## Importing Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,GlobalAveragePooling2D,BatchNormalization, Activation, Dropout,Input, Lambda
from tensorflow.keras.regularizers import l2,L1
from tensorflow.keras.initializers import he_normal
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.applications import Xception
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:

IMAGE_SIZE = 512
NUM_CLASSES = 4
# Set batch size
BATCH_SIZE = 8
# Set learning rate
LR = 1e-5
# Set epoch
EPOCHS = 70


## Data Loading, Preprocessing, and Generator Setup

##### Loading CSV file, adjusting image names, defining directories, preprocessing images, setting up a custom data generator class, and preparing data for training.

In [ ]:
# Load the CSV file with ground truth
labels_df = pd.read_csv('publish.csv')

# Adjust image names in labels_df to match the format without extension
labels_df['Image Name'] = labels_df['Image Name'].str.replace('.tif', '')

# Define paths to the directories with training and test images
train_image_dir = 'masks/train'
test_image_dir = 'masks/test'

# Get a list of all image files in the directories without extensions
train_image_files = [f[:-4] for f in os.listdir(train_image_dir) if f.endswith('.png')]
test_image_files = [f[:-4] for f in os.listdir(test_image_dir) if f.endswith('.png')]

# Filter labels_df to include only the images that exist in train_image_dir and test_image_dir
train_df = labels_df[labels_df['Image Name'].isin(train_image_files)].reset_index(drop=True)
test_df = labels_df[labels_df['Image Name'].isin(test_image_files)].reset_index(drop=True)

# Add the file extension .png to Image Name in train_df and test_df
train_df['Image Name'] = train_df['Image Name'].apply(lambda x: os.path.join(train_image_dir, x + '.png'))
test_df['Image Name'] = test_df['Image Name'].apply(lambda x: os.path.join(test_image_dir, x + '.png'))

# Optional: Save the filtered DataFrames to new CSV files if needed
train_df.to_csv('filtered_train_data.csv', index=False)
test_df.to_csv('filtered_test_data.csv', index=False)

# Function to read and preprocess an image
def load_and_preprocess_image(filepath):
    image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
    image = np.expand_dims(image, axis=-1)  # Add channel dimension for grayscale
    image = np.array(image, dtype=np.float32) / 255.0  # Normalize to [0, 1]
    return image

# Define DataGenerator class
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, images, labels, batch_size, target_size, n_classes, shuffle=True):
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.target_size = target_size
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.images) / self.batch_size))

    def __getitem__(self, index):
        batch_images = self.images[index * self.batch_size:(index + 1) * self.batch_size]
        batch_labels = self.labels[index * self.batch_size:(index + 1) * self.batch_size]
        return np.array(batch_images), tf.keras.utils.to_categorical(batch_labels, num_classes=self.n_classes)

    def on_epoch_end(self):
        if self.shuffle:
            indices = np.arange(len(self.images))
            np.random.shuffle(indices)
            self.images = self.images[indices]
            self.labels = self.labels[indices]

# Define target size
target_size = (IMAGE_SIZE, IMAGE_SIZE)

# Create generators
train_generator = DataGenerator(
    np.array([load_and_preprocess_image(filename) for filename in train_df['Image Name']]),
    train_df['MF-Grade'].values,
    batch_size=BATCH_SIZE, 
    target_size=target_size, 
    n_classes=NUM_CLASSES
)
test_generator = DataGenerator(
    np.array([load_and_preprocess_image(filename) for filename in test_df['Image Name']]),
    test_df['MF-Grade'].values,
    batch_size=BATCH_SIZE,
    target_size=target_size,
    n_classes=NUM_CLASSES,
    shuffle=False
)

# Now you can use train_generator and test_generator for model training
x_train, y_train = [], []
for batch_x, batch_y in train_generator:
    x_train.extend(batch_x)
    y_train.extend(batch_y)
x_train = np.array(x_train)
y_train = np.array(y_train)

print(len(x_train))


In [ ]:
# Prepare test data
x_test, y_test = [], []
for batch_x, batch_y in test_generator:
    x_test.extend(batch_x)
    y_test.extend(batch_y)
x_test = np.array(x_test)
y_test = np.array(y_test)
print("Number of test samples:", len(x_test))

## Class Distribution Visualization


##### Plotting a pie chart to visualize the distribution of classes in the training data.

In [ ]:
class_counts = np.sum(y_train, axis=0)

# Define class labels
class_labels = ['mf0', 'mf1', 'mf2', 'mf3']  # Update these labels based on your actual classes

# Plot the pie chart
plt.figure(figsize=(14, 6))
colors = ['#4285f4', '#ea4335', '#fbbc05', '#34a853']
plt.rcParams.update({'font.size': 14})
plt.pie(class_counts, labels=class_labels, colors=colors, autopct='%.1f%%', explode=(0.025, 0.025, 0.025, 0.025), startangle=30)
plt.show()

In [ ]:
def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True):
    initializer = he_normal(seed=90)
    x = Conv2D(filters=n_filters, 
               kernel_size=(kernel_size, kernel_size), 
               kernel_initializer=initializer, 
               padding='same',
               kernel_regularizer=l2(1e-5))(input_tensor)  # Adjust regularization strength as needed
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

## Model Definition

In [ ]:
def unet(num_classes=4):
    # Define input layer for grayscale images
    inputs = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 1), name="input_image")
    
    # Convert grayscale to RGB by duplicating the channel
    rgb_inputs = Lambda(lambda x: tf.image.grayscale_to_rgb(x))(inputs)
    
    encoder = Xception(input_tensor=rgb_inputs, include_top=False, weights="imagenet")
    
    # Freeze the layers 
    for layer in encoder.layers:
        layer.trainable = False

    encoder_output = encoder.layers[-1].output
    # Additional convolutional layers
    x = conv2d_block(encoder_output,n_filters=512, kernel_size=3,batchnorm=True)
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.7)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model  

# Instantiate the model
model = unet(num_classes=NUM_CLASSES)


In [ ]:
model.summary()

In [ ]:
# Set up early stopping and reduce learning rate on plateau
early_stopping = EarlyStopping(
    monitor='accuracy', 
    patience=10, 
    mode='max'
)

reduce_lr = ReduceLROnPlateau(
    monitor='accuracy', 
    factor=0.8,   
    patience=6, 
    min_lr=0.00000001,
)

In [ ]:
# Set up checkpoint for saving model weights
checkpoint_filepath = 'checkpoint.model.keras'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='accuracy',  # Monitor validation accuracy for saving the best model
    mode='max',  # Save the model with the highest validation accuracy
    save_best_only=True  # Only save the best model
)

## Model Compilation and Training

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
train_steps = len(x_train)//BATCH_SIZE

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator, epochs=EPOCHS, steps_per_epoch=train_steps,callbacks=[early_stopping, reduce_lr, model_checkpoint_callback])

## Training Loss and Accuracy Visualization

In [ ]:
plt.plot(history.history["loss"])
# plt.plot(history.history["val_loss"])
plt.title("Training Loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.show()
plt.plot(history.history["accuracy"])
# plt.plot(history.history["val_accuracy"])
plt.title("Training accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.show()

## Model Evaluation and Metrics Calculation

In [ ]:
from sklearn.metrics import accuracy_score

# Make predictions on the test data
test_predictions = model.predict(test_generator)

# Convert predictions to class labels
predicted_labels = np.argmax(test_predictions, axis=1)

# Get true labels from the test generator
true_labels = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Test accuracy: {accuracy * 100:.2f}%")

# Optional: Print classification report for detailed metrics
from sklearn.metrics import classification_report
print(classification_report(true_labels, predicted_labels, target_names=['Class 0', 'Class 1', 'Class 2', 'Class 3']))

## Confusion Matrix Visualization

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
cm = confusion_matrix(true_labels, predicted_labels)
sns.heatmap(cm, 
            annot=True,
            fmt='g', 
            xticklabels=['mf0', 'mf1', 'mf2', 'mf3'],
            yticklabels=['mf0', 'mf1', 'mf2', 'mf3'])
plt.xlabel('Prediction', fontsize=13)
plt.ylabel('Actual', fontsize=13)
plt.title('Confusion Matrix', fontsize=17)
plt.show()

## Saving the Model

In [ ]:
model.save('model_saves/classification_model.keras') 